# 1 - Imports

In [1]:
import sys
sys.path.append('/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/scripts/')

import argparse
import librosa
import librosa.display as ld
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from model_evaluator_2 import ModelEvaluator
from settings import MODEL_EVALUATOR_DEFAULT_SETTINGS

# 2 - Feature extractor class

In [2]:
default_params_dict = MODEL_EVALUATOR_DEFAULT_SETTINGS

default_params_dict['model_checkpoint_path'] = '/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/models/v1_tiny/v1_tiny_VGGNL_DoubleMHA_700.chkpt'
default_params_dict['test_clients'] = '/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/labels/test/voxceleb1_t_protocol/clients.ndx'
default_params_dict['test_impostors'] = '/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/labels/test/voxceleb1_t_protocol/impostors.ndx'
default_params_dict['dump_folder'] = './models_results/v1___'
default_params_dict['data_dir'] = '/home/usuaris/veussd/DATABASES/VoxCeleb/VoxCeleb1/test/'

default_params = argparse.Namespace(**default_params_dict)

model_evaluator_2 = ModelEvaluator(default_params)

Setting device...
Running on cuda device.
Device setted.
Setting random seed...
Random seed setted.


In [3]:
model_evaluator_2.load_checkpoint()
model_evaluator_2.load_checkpoint_params()
model_evaluator_2.load_network()

Loading checkpoint from /home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/models/v1_tiny/v1_tiny_VGGNL_DoubleMHA_700.chkpt


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

# 3 - Tests

In [ ]:
model_evaluator_2.load_data()

In [ ]:
for model_evaluator_2.batch_number, (input_1, input_2, label) in enumerate(model_evaluator_2.training_generator):
    print("-"*50)
    break